# Explaining maching learning
You can find in this notebook the exploratory data analysis done over the heart disease dataset, coupled with the simple machine learning algorithm used to predict the diseases.

## Plan
1. Introduction
  1. What are we going to explain here
    1. Explain AI/ML
    2. Not a tutorial, un exemple concret tout public de à quoi ça pourrait servir
    3. Essayer d'expliquer le taff d'un DS pour les gens qu'on connait et ceux interessés en general
  2. Why do we need to talk about AI in general
    1. Future
    2. Generic term needs to be addressed
    3. See the good side
2. What is a Data Scientist
  1. Work with data
  2. Data Analysis
  2. create models
  3. communicate
3. Example dataset
  1. Explaining the dataset and the goal
  2. A few statistics/plots
  3. Predicting the heart disease
  4. Explaining results
4. Communicating results to business/boss
  1. Presenting the example's results
  2. Another level of abstraction ?
5. (OPT) Cleaning codebase/Explaining why we would need another language
6. Conclusions, pointing to the notebook.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.utils.data import Dataset
from torch.utils.data import SequentialSampler, DataLoader, WeightedRandomSampler, BatchSampler
from sklearn import metrics
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
%config InlineBackend.figure_format = 'retina'
pd.set_option('display.max_columns', 500)

## Dataset fields description
1. age: age in years
2. sex: sex (1 = male; 0 = female)
3. cp: chest pain type
    - Value 1: typical angina
    - Value 2: atypical angina
    - Value 3: non-anginal pain
    - Value 4: asymptomatic
4. trestbps: resting blood pressure (in mm Hg on admission to the 
    hospital)
5. chol: serum cholestoral in mg/dl
6. fbs: (fasting blood sugar > 120 mg/dl)  (1 = true; 0 = false)
7. restecg: resting electrocardiographic results
    - Value 0: normal
    - Value 1: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV)
    - Value 2: showing probable or definite left ventricular hypertrophy by Estes' criteria
8. thalach: maximum heart rate achieved
9. exang: exercise induced angina (1 = yes; 0 = no)
10. oldpeak = ST depression induced by exercise relative to rest
11. slope: the slope of the peak exercise ST segment
    - Value 1: upsloping
    - Value 2: flat
    - Value 3: downsloping
12. ca: number of major vessels (0-3) colored by flourosopy
13. thal: 3 = normal; 6 = fixed defect; 7 = reversable defect
14. num: diagnosis of heart disease (angiographic disease status)
    - Value 0: < 50% diameter narrowing
    - Value 1: > 50% diameter narrowing
    (in any major vessel: attributes 59 through 68 are vessels)

In [ ]:
heart_df = pd.read_csv("../data/heart-disease/processed.cleveland.data", delimiter=",",
                       names=["age", "sex", "cp", "trestbps", "chol", "fbs", "restecg",
                              "thalach", "exang", "oldpeak", "slope", "ca", "thal", "num"])
heart_df = heart_df.rename(columns={"cp":"chest_pain",
                                    "thalach":"max_heart_rate",
                                    "oldpeak":"st_dep_induced",
                                    "ca":"num_maj_ves"})

heart_df['num_bin'] = heart_df['num'].apply(lambda x: 1 if x > 0 else 0)

heart_df = heart_df.replace('?', np.nan)
heart_df = heart_df.dropna(axis=0, how="any")

heart_df[['num_maj_ves', 'thal']] = heart_df[['num_maj_ves', 'thal']].astype('float')
heart_df= heart_df.reset_index(drop=True)

heart_df.drop("num", inplace=True, axis=1)
heart_df.head()

## Statistical exploratory analysis

In [ ]:
heart_df.describe()

### Plotting the dataset

In [ ]:
heart_df_plot = heart_df.copy()
heart_df_plot[["sex", "chest_pain", "fbs",
               "restecg", "exang", "slope",
               "num_maj_ves", "thal", "num_bin"]] = heart_df_plot[["sex", "chest_pain", "fbs", "restecg",
                                                                   "exang", "slope", "num_maj_ves", "thal", 
                                                                   "num_bin"]
                                                                 ].apply(lambda x: x.astype('category'))

fig, ax = plt.subplots(nrows=7, ncols=2, figsize=(10, 25))
ax = ax.reshape(-1)
plt.subplots_adjust(wspace=0.4, hspace=0.5)

for i, col in enumerate(heart_df_plot.columns):
    if heart_df_plot[col].dtype.name == "category":
        sns.countplot(x=col, data=heart_df_plot, ax=ax[i])
        ax[i].set_ylabel("count")
    else:
        sns.kdeplot(heart_df_plot[col], ax=ax[i])
    ax[i].set_xlabel(col)

In [ ]:
# Standardizing dataset
standardizing_heart_df = heart_df.copy()
scaler = preprocessing.StandardScaler()
# Checking out the scattered matrix
standardizing_heart_df.iloc[:, :-1] = scaler.fit_transform(heart_df.drop(['num_bin'], axis=1))
pd.plotting.scatter_matrix(standardizing_heart_df, alpha=0.3, figsize=(15,15), diagonal='kde');

# Checking our the heatmap of correlations
plt.figure(figsize=(15, 15))
correlation = standardizing_heart_df.corr(min_periods=10)
heatmap = sns.heatmap(correlation, annot=True, linewidths=0, vmin=-1, cmap="RdBu_r")
plt.show()

### Training and testing splits

In [ ]:
class HeartDataset(Dataset):
    def __init__(self, df, train=True):
        self.heart_df = df
        self.number_samples = len(df)
        self.train = train
    
    def __len__(self):
        return self.number_samples
    
    def __getitem__(self, idx):
        if not self.train:
            idx = torch.LongTensor([idx])
        features = self.heart_df.iloc[idx.item(), :-1]
        binary_class = self.heart_df.iloc[idx.item(), -1]
        return torch.FloatTensor(features.values), binary_class

In [ ]:
train_size = 0.8
train_sample_number = int(train_size*len(heart_df))

weight_one = 1 - sum(heart_df.iloc[:,-1]) / len(heart_df)
weight = [weight_one if c == 1 else 1 - weight_one for c in heart_df.iloc[:,-1] ]

total_dataset = HeartDataset(heart_df)
sampler = WeightedRandomSampler(weight, len(total_dataset), replacement=False)
batch_sampler = BatchSampler(sampler, train_sample_number, drop_last=False)

train_set, test_set = ([i.item() for i in cl] for cl in batch_sampler)
train_df = heart_df.iloc[train_set]
test_df = heart_df.iloc[test_set]
print("training set: {} samples, {:.2f}% of which belong to the first class".format(len(train_df), train_df.num_bin.value_counts().values[0] / len(train_df) * 100))
print("testing set: {} samples, {:.2f}% of which belong to the first class".format(len(test_df), test_df.num_bin.value_counts().values[0] / len(test_df) * 100))

## Random Forest Classifier
# DO NOT STANDARDIZE FOR RFC, NO USE FOR IT, PROBLEMS WITH ANALYZING RESULTS IF WE DO
Training with 5-fold cross validation to find the best parameters, and computing the accuracy, precision/recall/f1score.

In [ ]:
# FINAL CELL FOR RFC START

# The goal is to predict if a person has a heart disease or not.
# Thus we have decided to try a random forest classifier
# Has we want to do a kind of diagnosis thus we want to optimize the recall
# Because it is more dangerous to miss a heart disease than to over diagnosis
train_features, train_targets = train_df.iloc[:,:-1], train_df.iloc[:,-1]
test_features, test_targets = test_df.iloc[:,:-1], test_df.iloc[:,-1]

# classic random forest classifier
rfc = RandomForestClassifier()

# the parameters we want to test in order to find the best combinations
parameters = {'n_estimators': list(range(5,40,5)), 'max_features':list(range(2,10)), 'max_depth': list(range(2,10))}
# the score used to optimize the classifier (recall here)
recall_score = metrics.make_scorer(metrics.recall_score)
# then we can make the grid search cross validation (5-fold)
grid_search = GridSearchCV(rfc, parameters, scoring=recall_score, cv=5)
# we can fit the grid search object with our training set
grid_search_fit = grid_search.fit(train_features, train_targets)
# then we can deduce the best classifier (the best parameters combination)
best_rfc = grid_search_fit.best_estimator_
# and we make the predictions for our testing set
best_predict = best_rfc.predict(test_features)

print("Best parameters : ", grid_fit.best_params_)
# accuracy = tp + tn / total, pas bon si desequilibre des labels 
# (le classifier peut donner toujours le même label)
print("\naccuracy :", metrics.accuracy_score(test_targets, best_predict))
# recall = tp / tp + fn (total de vrai malade du test set) => très important ici,
# car on veut trouver le plus de malade possible (plus grave de rater un malade, que d'en diagnostiquer trop)
print("recall :", metrics.recall_score(test_targets, best_predict))
# precision = tp / tp + fp (total de malade que le classifier a trouvé) => moins important ici,
# car determine combien de diagnostiqués malades, sont vraiment malades 
print("precision :", metrics.precision_score(test_targets, best_predict))
# f1 score = 2 * (precision * recall) / (precision + recall)
print("f1 score :", metrics.f1_score(test_targets, best_predict), "\n")

# importance of each features
for i, feat in enumerate(best_rfc.feature_importances_):
    print("{} : {}".format(list(heart_df)[i], feat))

In [ ]:
#https://towardsdatascience.com/how-to-visualize-a-decision-tree-from-a-random-forest-in-python-using-scikit-learn-38ad2d75f21c

# Visualize the decision tree
from sklearn.datasets import load_iris
iris = load_iris()

# Extract single tree of our best random forest classifier
estimator = best_rfc.estimators_[0]

from sklearn.tree import export_graphviz
# Export as dot file
export_graphviz(estimator, out_file='tree.dot', 
                feature_names = list(train_df.iloc[:, :-1]),
                class_names = iris.target_names,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Convert to png using system command (requires Graphviz)
from subprocess import call
call(['dot', '-Tpng', 'tree.dot', '-o', 'tree.png', '-Gdpi=600'])

# Display in jupyter notebook
from IPython.display import Image
Image(filename = 'tree.png')

## Deep Learning
# STANDARDIZE TRAINING SET AND TESTING SET THE SAME WAY TO BE SURE WE GET THE SAME THING

In [ ]:
# Creating a validation set
train_nn_size = 0.8
train_nn_sample_number = int(train_nn_size*len(train_df))

weight_one = 1 - sum(train_df.iloc[:,-1]) / len(train_df)
weight = [weight_one if c == 1 else 1 - weight_one for c in train_df.iloc[:,-1] ]

# Standardizing training set
standardized_nn_train_df = train_df.copy()
scaler = preprocessing.StandardScaler()
scaler.fit(train_df.drop(['num_bin'], axis=1))
standardized_nn_train_df.iloc[:, :-1] = scaler.transform(train_df.drop(['num_bin'], axis=1))

train_nn_set = HeartDataset(standardized_nn_train_df)
sampler = WeightedRandomSampler(weight, len(train_nn_set), replacement=False)
batch_sampler = BatchSampler(sampler, train_nn_sample_number, drop_last=False)

train_set, valid_set = ([i.item() for i in cl] for cl in batch_sampler)
train_nn_df = train_df.iloc[train_set]
valid_df = train_df.iloc[valid_set]
print("training set: {} samples, {:.2f}% of which belong to the first class".format(len(train_nn_df), train_nn_df.num_bin.value_counts().values[0] / len(train_nn_df) * 100))
print("validation set: {} samples, {:.2f}% of which belong to the first class".format(len(valid_df), valid_df.num_bin.value_counts().values[0] / len(valid_df) * 100))
print("testing set: {} samples, {:.2f}% of which belong to the first class".format(len(test_df), test_df.num_bin.value_counts().values[0] / len(test_df) * 100))

In [ ]:
batch_size = 8

train_dataset = HeartDataset(train_nn_df)
valid_dataset = HeartDataset(valid_df, train=False)

# Do not forget to apply the same preprocessing to the testing set
standardizing_test_df = test_df.copy()
standardizing_test_df.iloc[:, :-1] = scaler.transform(test_df.drop(['num_bin'], axis=1))
test_dataset = HeartDataset(standardizing_test_df, train=False)

weight_one = 1 - sum(train_nn_df.iloc[:,-1]) / len(train_nn_df)
weight = [weight_one if c == 1 else 1 - weight_one for c in train_nn_df.iloc[:,-1] ]

sampler = WeightedRandomSampler(weight, len(train_dataset))
b_sampler = BatchSampler(sampler, batch_size, True)

train_dataset_loader = DataLoader(dataset=train_dataset,  num_workers=0, batch_sampler=b_sampler )
valid_dataset_loader = DataLoader(dataset=valid_dataset, batch_size=batch_size)
test_dataset_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)

In [ ]:
def train_model(model, train_dataset_loader, valid_dataset_loader, nb_epochs):
    """
    Will train the model and retain the training and validation losses at each epoch
    """
    training_losses, validation_losses = [], []
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    model.train()
    for e in range(nb_epochs):
        train_sum_loss = 0
        valid_sum_loss = 0
        
        for feat, c in train_dataset_loader:
            output = model(feat)
            loss = criterion(output, c)
            model.zero_grad()
            loss.backward()
            optimizer.step()
            train_sum_loss += loss.item()
            
        for feat, c in valid_dataset_loader:
            output = model(feat)
            loss = criterion(output, c)
            valid_sum_loss += loss.item()
            
        if (e+1) % 100 == 0:
            print("epoch: {:4d}/{:4d} | training loss: {:3.4f} | validation loss: {:3.4f}".format(
                e+1, nb_epochs, normalized_train_loss, normalized_valid_loss))
            
        if (e+1) % 5 == 0:
            normalized_train_loss = train_sum_loss / len(train_dataset_loader)
            normalized_valid_loss = valid_sum_loss / len(valid_dataset_loader)
            training_losses.append(normalized_train_loss)
            validation_losses.append(normalized_valid_loss)
        
    plt.figure(figsize=(12, 8))
    plt.title("Training and validation losses")
    plt.plot(training_losses, 'red', label='training loss')
    plt.plot(validation_losses, 'green', label='validation loss')
    plt.legend(loc='best')
    plt.show()

def compute_nb_errors(model, dataset_loader):
    model.eval()
    nb_data_errors = 0
    #nb_recall_errors = 0
    #nb_pos = 0
    
    for feat, c in dataset_loader:
        output = model(feat)
        _, predicted_classes = torch.max(output, 1)
        nb_data_errors += sum(abs(predicted_classes - c)).item()
        #DO RECALL
        #nb_recall_errors += sum(predicted_classes == 1 & c == 1).item()
        #nb_pos += sum(c == 1).item()

    accuracy = (1 - nb_data_errors/ (len(dataset_loader)*batch_size)) *100
    #recall = nb_recall_errors / nb_pos * 100
    return accuracy#, recall


In [ ]:
drop = 0.6
model = torch.nn.Sequential(torch.nn.Linear(13,200),
                            torch.nn.Dropout(drop),
                            torch.nn.ReLU(),
                            torch.nn.Linear(200,100),
                            torch.nn.Dropout(drop),
                            torch.nn.ReLU(),
                            torch.nn.Linear(100,50),
                            torch.nn.Dropout(drop),
                            torch.nn.ReLU(),
                            torch.nn.Linear(50,2))
train_model(model, train_dataset_loader, valid_dataset_loader, 1200)
print("With dropout:", drop)
print("training set accuracy: {:2.2f}%".format(compute_nb_errors(model, train_dataset_loader)))
print("validation set accuracy: {:2.2f}%".format(compute_nb_errors(model, valid_dataset_loader)))
print("testing set accuracy: {:2.2f}%".format(compute_nb_errors(model, test_dataset_loader)))